# Pre

Import libraries

In [ ]:

!pip install torch
!pip install transformers

In [ ]:
import pandas as pd
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import numpy as np
import os, re, string
import torch
import re
import torch.nn as nn
import logging
from transformers import AutoTokenizer,  PhobertTokenizer

In [ ]:
# connect gg drive
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/Colab Notebooks/Challenge'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Colab Notebooks/Challenge


In [ ]:
df = pd.read_json("ise-dsc01-train.json")
df = df.transpose()
df.head(10)


,context,claim,verdict,evidence,domain
7125,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Ngoài việc không giới hạn mức lương, công ty c...",SUPPORTED,"Công ty cũng có chế độ đãi ngộ tốt, có lương t...",giao-duc
7126,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Thành lập năm 2016, phát triển phần mềm, tổ ch...",SUPPORTED,"FABA Technology thành lập năm 2016, chuyên cun...",giao-duc
7127,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Nhằm phát triển đội ngũ, FABA thường tổ chức n...",SUPPORTED,"Bên cạnh đó, FABA thường xuyên tổ chức những d...",giao-duc
7128,"Đối với các nhà khoa học, trí thức, điều kiện ...",Sở dĩ vị trí kỹ sư phần mềm có mức lương cao n...,SUPPORTED,"Theo Adeco Việt Nam, mức lương cao nhất đối vớ...",giao-duc
7129,"Đối với các nhà khoa học, trí thức, điều kiện ...","Theo Adeco Việt Nam, mức lương cao nhất đối vớ...",SUPPORTED,"Trong khi đó, theo báo cáo ""Thị trường nhân lự...",giao-duc
7130,"Đối với các nhà khoa học, trí thức, điều kiện ...",Việc ủng hộ phương án vừa làm việc ở nước ngoà...,SUPPORTED,Tiến sĩ Liêm cũng ủng hộ phương án vừa làm việ...,giao-duc
7131,"Thầy Anh kể, từ nhiều năm trước, thông tin về ...",Một học sinh tâm sự với thầy rằng không dám gó...,SUPPORTED,Học sinh này tâm sự rằng các em rất muốn được ...,giao-duc
7132,"Thầy Anh kể, từ nhiều năm trước, thông tin về ...","Khi tiếp nhận, nếu nội dung phản ánh nằm ngoài...",SUPPORTED,"Nhận tin báo, thầy Anh chuyển nội dung tới côn...",giao-duc
7133,"Thầy Anh kể, từ nhiều năm trước, thông tin về ...","Ví dụ, năm 2020 một học sinh tại THCS Quỳnh Ph...",SUPPORTED,"Hơn hai năm công bố số điện thoại, vị hiệu trư...",giao-duc
7134,"PGS.TS Nguyễn Quang Liệu, hiệu trưởng trường T...",Điểm xét tuyển là tổng điểm các môn thi theo t...,SUPPORTED,Trường không cộng điểm ưu tiên trong tuyển sin...,giao-duc


In [ ]:
test_df = pd.read_json("ise-dsc01-public-test-offcial.json")
test_df = test_df.transpose()

test_df.head(10)

,context,claim
29706,"Ngày 13/3, PGS.TS Phạm Cẩm Phương, Giám đốc Tr...",Nên kiểm tra gene trẻ sơ sinh khi gia đình có ...
32454,"STEM bao gồm lĩnh vực khoa học (Science), công...","mba tập trung vào kỹ năng kinh doanh, quản lý ..."
41947,Trong khi tranh cãi còn chưa dứt về số tài liệ...,Cuộc tranh cãi còn chưa dứt về số tài liệu mật...
47909,"Nằm trên đường Nguyễn Biểu, quận 5, tiệm há cả...","Tiệm há cảo của anh Hùng có 6 vị nhân, được là..."
46798,"Năm nay, thời tiết nắng nhiều nhưng mưa ít khi...","Cách trung tâm Thủ đô chừng 100km, công viên T..."
47255,Đây là năm thứ 2 cuộc thi Sáng kiến Khoa học (...,Yêu cầu giải pháp/sản phẩm dự thi được mô tả b...
41443,Mì sụa là một trong những món ăn truyền thống ...,Món ăn truyền thống của người Hoa là mì sụa
39120,Trong kỳ nghỉ lễ kéo dài 5 ngày dịp 30/4 - 1/5...,Trong các ngày cuối tuần từ 14/4 đến dịp lễ 30...
32538,Nguyễn Ngọc Thiện là học sinh lớp 7 tại trường...,thiện đã chia sẻ kinh nghiệm học tập với hơn 1...
46995,Đây là năm thứ hai VnExpress tổ chức cuộc thi ...,Ngoài ra cuộc thi cũng chọn và trao cho tác gi...


In [ ]:
def clean_text(example):
  text = example
  text = re.sub(r'[^\w\s]', '', text)
  text = re.sub(r'[ \n]', ' ', text)
  text = re.sub(r' +', ' ', text).strip()
  text = text.lower()
  return text


In [ ]:
verdicts = df['verdict'].unique().tolist()
num_labels = len(verdicts)
def to_category_vector(label):
  return verdicts.index(label)

# Show example
print(to_category_vector("NEI"))
print(num_labels)

1
3


In [ ]:
df['evidence'].head(20)

7125    Công ty cũng có chế độ đãi ngộ tốt, có lương t...
7126    FABA Technology thành lập năm 2016, chuyên cun...
7127    Bên cạnh đó, FABA thường xuyên tổ chức những d...
7128    Theo Adeco Việt Nam, mức lương cao nhất đối vớ...
7129    Trong khi đó, theo báo cáo "Thị trường nhân lự...
7130    Tiến sĩ Liêm cũng ủng hộ phương án vừa làm việ...
7131    Học sinh này tâm sự rằng các em rất muốn được ...
7132    Nhận tin báo, thầy Anh chuyển nội dung tới côn...
7133    Hơn hai năm công bố số điện thoại, vị hiệu trư...
7134    Trường không cộng điểm ưu tiên trong tuyển sin...
7135    Điểm xét tuyển vào lớp này là tổng ba môn chun...
7136    Điểm xét tuyển vào lớp này là tổng ba môn chun...
7137    Thí sinh chỉ trúng tuyển một nguyện vọng cao n...
7138    Các ngành thuộc khối sức khỏe có mức điểm chuẩ...
7139    Các ngành thuộc khối sức khỏe có mức điểm chuẩ...
7140     Chỉ tiêu, tổ hợp xét tuyển từng ngành như sau...
7141    Năm ngoái, điểm chuẩn Học viện Ngân hàng theo ...
7142    TS Trầ

In [ ]:
df['evidence'] = df['evidence'].fillna('Không có chứng cứ')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26967 entries, 7125 to 48031
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   context   26967 non-null  object
 1   claim     26967 non-null  object
 2   verdict   26967 non-null  object
 3   evidence  26967 non-null  object
 4   domain    26967 non-null  object
dtypes: object(5)
memory usage: 2.2+ MB


In [ ]:

# ctxDF = df['context'].apply(text_to_sentences)
# claimDF = df['claim'].apply(text_to_sentences)
ctxDF = df['context'].apply(clean_text)
claimDF = df['claim'].apply(clean_text)
evidenceDF = df['evidence'].apply(clean_text)

testCtxDF = test_df['context'].apply(clean_text)
testClaimDF = test_df['claim'].apply(clean_text)

In [ ]:
verDF = df['verdict'].apply(to_category_vector)
verDF

7125     0
7126     0
7127     0
7128     0
7129     0
        ..
48027    2
48028    2
48029    2
48030    2
48031    2
Name: verdict, Length: 26967, dtype: int64

In [ ]:
contexts = ctxDF.values.tolist()
claims = claimDF.values.tolist()
verdicts = verDF.values.tolist()
evidences = evidenceDF.values.tolist()

In [ ]:
len(verdicts)

26967

# Post

In [ ]:
# train_contexts, test_contexts, train_claims, test_claims, train_verdicts, test_verdicts,  = train_test_split(contexts, claims,verdicts, test_size=0.2, random_state=42)
train_contexts =contexts
train_claims = claims
train_verdicts = verdicts
train_evidences = evidences

test_contexts = testCtxDF.values.tolist()
test_claims = testClaimDF.values.tolist()

In [ ]:


tokenizer = PhobertTokenizer.from_pretrained('vinai/phobert-base')

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

In [ ]:
train_encoded_inputs = tokenizer(train_contexts,train_claims,
                           add_special_tokens=True,
                           truncation=True,
                           padding='max_length', max_length=64, return_tensors="pt")


In [ ]:
test_encoded_inputs = tokenizer(test_contexts,test_claims,
                           add_special_tokens=True,
                           truncation=True,
                           padding='max_length', max_length=64, return_tensors="pt")


In [ ]:

# Mã hóa nhãn (verdict) thành các giá trị số nguyên
# label_map = {"SUPPORTED": 0, "NEI": 1, "REFUTED": 2}
# labels = [label_map[label] for label in verdicts]

train_input_ids = train_encoded_inputs["input_ids"]
train_attention_mask = train_encoded_inputs["attention_mask"]
train_labels = torch.tensor(train_verdicts)


In [ ]:
test_input_ids = test_encoded_inputs["input_ids"]
test_attention_mask = test_encoded_inputs["attention_mask"]


In [ ]:
torch.save(train_input_ids, "train_input_ids.pt")
torch.save(train_attention_mask, "train_attention_mask.pt")
torch.save(train_labels, "train_labels.pt")

In [ ]:
torch.save(test_input_ids, "test_input_ids.pt")
torch.save(test_attention_mask, "test_attention_mask.pt")

In [ ]:
train_combined_data = [context + ' ' + claim for context, claim in zip(train_contexts, train_claims)]
test_combined_data = [context + ' ' + claim for context, claim in zip(test_contexts, test_claims)]

In [ ]:
# Encode claim and context using TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()
train_tfidf_matrix = tfidf_vectorizer.fit_transform(train_combined_data)
test_tfidf_matrix = tfidf_vectorizer.fit_transform(test_combined_data)
# Apply SVD to reduce dimensionality of TF-IDF matrix
svd = TruncatedSVD(n_components=64)
train_svd_matrix = svd.fit_transform(train_tfidf_matrix)
test_svd_matrix = svd.fit_transform(test_tfidf_matrix)

In [ ]:
train_svd_matrix= torch.tensor(train_svd_matrix)
test_svd_matrix = torch.tensor(test_svd_matrix)

In [ ]:
torch.save(train_svd_matrix, "train_svd_matrix.pt")
torch.save(test_svd_matrix, "test_svd_matrix.pt")